<a href="https://colab.research.google.com/github/AbishekArumugam1/TataIQ-GenAI-Data-Analytics-Simulation/blob/main/Geldium_EDA_Report_Abishek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geldium EDA Report
Course: Gen AI Powered Data Analytics — Tata iQ  
Author: Abishek Arumugam S

**Objective:** Explore and clean the dataset so it's ready for modeling.


In [1]:
# Step 0: Load libraries we will use
import pandas as pd        # read and manipulate tables
import numpy as np         # math helpers
import matplotlib.pyplot as plt   # plots
import seaborn as sns             # nicer plots

# Make plots a good size
plt.rcParams['figure.figsize'] = (10,5)

print("Ready! pandas version:", pd.__version__)


Ready! pandas version: 2.2.2


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import pandas as pd

filename = list(uploaded.keys())[0]
print("Loaded file:", filename)

# Read Excel file
df = pd.read_excel(filename)

# Display the first few rows
df.head()


In [ ]:
import pandas as pd

filename = list(uploaded.keys())[0]
excel_file = pd.ExcelFile(filename)

print("Available sheets:")
print(excel_file.sheet_names)


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import pandas as pd
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename, encoding_errors='ignore')
df.head()


In [ ]:
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])


In [ ]:
df.info()   # shows columns, non-null counts, and data types


In [ ]:
df.describe().T   # mean, std, min, max, quartiles for numeric columns


In [ ]:
missing_count = df.isnull().sum()
missing_pct = (df.isnull().mean() * 100).round(2)
pd.concat([missing_count, missing_pct], axis=1, keys=['missing_count','missing_pct']).sort_values('missing_count', ascending=False)


In [ ]:
dups = df.duplicated().sum()
print("Duplicate rows found:", dups)


In [ ]:
df.head(10)


In [ ]:
import numpy as np


In [ ]:
# Replace common missing markers with actual NaN
df.replace(['?', 'NA', 'N/A', 'na', 'nil', 'None', '--', ''], np.nan, inplace=True)


In [ ]:
df.isnull().sum()


In [ ]:
# Replace common text markers with actual NaN
df.replace(['?', 'NA', 'N/A', 'na', 'nil', 'None'], np.nan, inplace=True)


In [ ]:
missing_count = df.isnull().sum()
missing_pct = (df.isnull().mean() * 100).round(2)
pd.concat([missing_count, missing_pct], axis=1, keys=['missing_count','missing_pct']).sort_values('missing_count', ascending=False)


In [ ]:
# Replace 'date_col' with your actual column name, e.g. 'order_date'
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    print("Converted 'date' column to datetime. Nulls after conversion:", df['date'].isnull().sum())


In [ ]:
num_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
print("Numeric columns:", num_cols)
print("Categorical columns:", cat_cols)


In [ ]:
df[num_cols].hist(bins=20, figsize=(15, 10))
plt.suptitle("Histograms for numeric columns")
plt.show()


In [ ]:
for col in num_cols:
    plt.figure(figsize=(8,2))
    sns.boxplot(x=df[col].dropna())
    plt.title(col)
    plt.show()


In [ ]:
# 1 — Sanity checks
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
from IPython.display import display

print("Variables in this session:", list(globals().keys())[:120])
print("\nDoes 'df' exist in memory? ->", 'df' in globals())
if 'df' in globals():
    print("DataFrame shape:", getattr(df, "shape", None))
    display(df.head(5))
    print("\nColumn names and dtypes:")
    display(df.dtypes)
else:
    print("No DataFrame named 'df' found. Upload and load the CSV first (use files.upload() then pd.read_csv).")


In [ ]:
# 2 — safe creation of num_cols and cat_cols
num_cols = df.select_dtypes(include=['number']).columns.tolist()
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()

print("Numeric columns (count={}):".format(len(num_cols)), num_cols)
print("Categorical columns (count={}):".format(len(cat_cols)), cat_cols)


In [ ]:
# 3 — try converting columns that look numeric but are objects
possible_nums = [c for c in df.columns if df[c].dtype == 'object']
converted = []
for c in possible_nums:
    # try coercion
    tmp = pd.to_numeric(df[c].str.replace(',',''), errors='coerce') if df[c].dtype == 'object' else pd.to_numeric(df[c], errors='coerce')
    non_null_ratio = tmp.notnull().mean()
    if non_null_ratio > 0.6:   # heuristic: more than 60% convertible
        df[c] = tmp
        converted.append(c)

if converted:
    print("Converted these object columns to numeric (heuristic):", converted)
else:
    print("No object columns converted automatically.")
# Recompute lists
num_cols = df.select_dtypes(include=['number']).columns.tolist()
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
print("Numeric columns now:", num_cols)


In [ ]:
# 4 — Histograms (safe)
if not num_cols:
    print("No numeric columns to plot histograms for.")
else:
    # limit plots if many numeric columns
    max_plots = 12
    cols_to_plot = num_cols[:max_plots]
    print(f"Plotting histograms for first {len(cols_to_plot)} numeric columns (of {len(num_cols)} total).")
    df[cols_to_plot].hist(bins=20, figsize=(15, 3 * ((len(cols_to_plot)+3)//4)))
    plt.suptitle("Histograms for numeric columns")
    plt.tight_layout()
    plt.show()


In [ ]:
# 5 — Boxplots (one-by-one)
if not num_cols:
    print("No numeric columns to plot boxplots for.")
else:
    for col in num_cols:
        try:
            plt.figure(figsize=(8,2))
            sns.boxplot(x=df[col].dropna())
            plt.title(f"Boxplot: {col}")
            plt.show()
        except Exception as e:
            print(f"Failed plotting boxplot for {col}: {e}")


In [ ]:
# 6 — categorical counts
if not cat_cols:
    print("No categorical columns detected.")
else:
    for col in cat_cols:
        try:
            print("----", col, "----")
            display(df[col].value_counts(dropna=False).head(10))
            plt.figure(figsize=(8,4))
            df[col].value_counts().nlargest(10).plot(kind='bar')
            plt.title(f"Top values: {col}")
            plt.xlabel(col)
            plt.ylabel("count")
            plt.show()
        except Exception as e:
            print(f"Failed for column {col}: {e}")


In [ ]:
# 7 — correlation heatmap
if len(num_cols) <= 1:
    print("Not enough numeric columns for correlation (need 2+). Numeric columns:", num_cols)
else:
    corr = df[num_cols].corr()
    plt.figure(figsize=(min(12, 1+len(num_cols)), min(10, 1+len(num_cols))))
    sns.heatmap(corr, annot=True, fmt=".2f", center=0)
    plt.title("Correlation between numeric columns")
    plt.show()


In [ ]:
for col in num_cols:
    plt.figure(figsize=(8,2))
    sns.boxplot(x=df[col].dropna())
    plt.title(col)
    plt.show()


In [ ]:
for col in cat_cols:
    print("----", col, "----")
    display(df[col].value_counts().head(10))


In [ ]:
if len(num_cols) > 1:
    corr = df[num_cols].corr()
    sns.heatmap(corr, annot=True, fmt=".2f", center=0)
    plt.title("Correlation between numeric columns")
    plt.show()
else:
    print("Not enough numeric columns for correlation.")


In [ ]:
# Numeric fill
for c in num_cols:
    if df[c].isnull().sum() > 0:
        df[c] = df[c].fillna(df[c].median())

# Categorical fill
for c in cat_cols:
    if df[c].isnull().sum() > 0:
        if not df[c].mode().empty:
            df[c] = df[c].fillna(df[c].mode().iloc[0])
        else:
            df[c] = df[c].fillna('Unknown')

print("Missing values after simple filling:")
display(df.isnull().sum().sort_values(ascending=False).head(20))


In [ ]:
def cap_iqr(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return series.clip(lower, upper)

for c in num_cols:
    df[c] = cap_iqr(df[c])
print("Applied IQR capping to numeric columns.")


In [ ]:
if 'price' in df.columns and 'quantity' in df.columns:
    df['total_value'] = df['price'] * df['quantity']
    print("Created 'total_value' feature.")


In [ ]:
cleaned_name = 'geldium_cleaned_by_abishek.csv'
df.to_csv(cleaned_name, index=False)
from google.colab import files
files.download(cleaned_name)   # will trigger a browser download


In [ ]:
# Try to run a small check
print("Final shape:", df.shape)
print("Any missing?:", df.isnull().any().any())
print("Sample rows:")
display(df.head())
